In [ ]:
#import sys
#sys.executable
#!/anaconda3/bin/python -m pip install rfcde==0.3.1

In [2]:
import rfcde
from matplotlib import pyplot as plt
#from cdetools.cde_loss import cde_loss
import numpy as np
import pandas as pd 

import seaborn as sns
import matplotlib.patches as patches

from scipy.spatial import ConvexHull, convex_hull_plot_2d

from shapely.geometry import Polygon
from shapely.geometry import MultiPoint

from sklearn.model_selection import train_test_split
import itertools

plt.rcParams["figure.figsize"] = (10,8)

In [6]:
df18 = pd.read_csv('modeldata2018.csv')
df19 = pd.read_csv('modeldata2019.csv')
df20 = pd.read_csv('modeldata_2020.csv')

train = pd.concat([df18,df19], axis=0)
test = df20

In [7]:
train2 = pd.read_csv('train.csv')
test2 = pd.read_csv('test.csv')

In [8]:
train2 = train2[['gameId', 'playId', 'frameId','prob_tackle' ]]
train = train[['gameId', 'playId', 'frameId','kick_area','return_area','carrier_area','return_yds_remaining','path_frechet' ]]

train_ = train.merge(train2, on = ['gameId', 'playId', 'frameId'])

test2 = test2[['gameId', 'playId', 'frameId','prob_tackle' ]]
test = test[['gameId', 'playId', 'frameId','kick_area','return_area','carrier_area','return_yds_remaining', 'path_frechet' ]]

test_ = test.merge(test2, on = ['gameId', 'playId', 'frameId'])

In [9]:
train = train_[0:2000]
#test = test[(test.gameId ==2019090500 )&(test.playId ==125)]
test = test_

z_train = train['return_yds_remaining'].values.reshape(-1,1)
x_train = train[['kick_area','return_area','carrier_area', 'path_frechet'  ]].values


z_test = test['return_yds_remaining'].values.reshape(-1,1)
x_test = test[['kick_area','return_area','carrier_area', 'path_frechet' ]].values

In [11]:
n_trees = 100
mtry = 1
node_size = 1

n_basis = 30
basis_system = 'cosine'

model = rfcde.RFCDE(n_trees=n_trees, mtry=mtry, node_size=node_size, n_basis=n_basis, 
                    basis_system=basis_system)

model.train(x_train, z_train)

In [12]:
n_grid = int(len(x_test))
bandwidth = 0.5
z_grid = np.linspace(0, 100, n_grid)
cde_test = model.predict(x_test, z_grid, bandwidth=bandwidth)

In [13]:
cde_test

array([[0.02873534, 0.02877916, 0.02882271, ..., 0.        , 0.        ,
        0.        ],
       [0.02923711, 0.02927817, 0.02931895, ..., 0.        , 0.        ,
        0.        ],
       [0.02910983, 0.02915072, 0.02919132, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.11336539, 0.11358665, 0.11380764, ..., 0.        , 0.        ,
        0.        ],
       [0.11053778, 0.11074836, 0.11095849, ..., 0.        , 0.        ,
        0.        ],
       [0.10934277, 0.10954943, 0.10975562, ..., 0.        , 0.        ,
        0.        ]])

In [14]:
cond_mean = model.predict_mean(x_test)
cond_median = model.predict_quantile(x_test, 0.5)
cond_mode = z_grid[np.argmax(cde_test, axis=1)]

In [15]:
preds = pd.DataFrame(cond_mode)
preds.columns = ['cond_mode']

preds2 = pd.DataFrame(cond_median)
preds2.columns = ['cond_median']

In [16]:
preds

,cond_mode
0,9.144489
1,9.162624
2,9.158090
3,9.144489
4,9.121821
...,...
22053,0.729927
22054,0.743528
22055,0.743528
22056,0.729927


In [19]:
df = pd.concat([test, preds, preds2], axis=1)

df.to_csv('results.csv')

In [22]:
df['diff'] = df['return_yds_remaining'] - df['cond_median']

In [29]:
gb = df.groupby(['gameId', 'playId'])['diff'].sum().reset_index(name='total_diff')

In [54]:
print(np.mean(df[(df.return_yds_remaining > 30)].groupby(['playId'])['diff'].mean()))
print(np.mean(df[(df.return_yds_remaining > 0)&(df.return_yds_remaining < 10)].groupby(['playId'])['diff'].mean()))
print(np.mean(df[(df.return_yds_remaining > 10)&(df.return_yds_remaining < 20)].groupby(['playId'])['diff'].mean()))





37.32659839835031
-2.395970761079905
5.153201634086938


In [50]:
np.mean(df[(df.return_yds_remaining > 0)&(df.return_yds_remaining < 10)].groupby(['playId'])['diff'].sum()/len(df[(df.return_yds_remaining > 20)&(df.return_yds_remaining < 30)].groupby(['playId'])['diff'].sum()))


-1.2145648956264359

In [57]:
print(np.mean(df.groupby(['playId'])['diff'].mean()))


-0.28811211289497357
